In [3]:
import nidaqmx

In [4]:
# Initializing nidaqmx system
system = nidaqmx.system.System.local()

In [5]:
# Printing out cards in chassis
for device in system.devices:
    print(device.product_type, device.name, device.serial_num)

PXIe-6341 PXI1Slot2 27490011


In [6]:
for device in system.devices:
    print(device, device.do_max_rate)

Device(name=PXI1Slot2) 1000000.0


In [8]:
import time
device = 'PXI1Slot2'
ao_channel = "ao1"
with nidaqmx.Task() as ao_task:
    ao_task.ao_channels.add_ao_voltage_chan(f"{device}/{ao_channel}")
    ao_task.write(5.0)
    ao_task.start()
    ao_task.stop()

time.sleep(5)

with nidaqmx.Task() as ao_task:
    ao_task.ao_channels.add_ao_voltage_chan(f"{device}/{ao_channel}")
    ao_task.write(0)
    ao_task.stop()

In [59]:
# Streaming variable setup

from random import sample
from nidaqmx.constants import AcquisitionType, RegenerationMode, WriteRelativeTo
import numpy as np
import threading
import time
import nidaqmx

device = 'PXI1Slot6'
ao_channel = "ao1"
sample_rate = 1e6
buffer_seconds = 5
num_channels = 1
buffer_size = int(sample_rate * buffer_seconds)
half_buffer = buffer_size//2

In [61]:
def continuous_waveform_streaming():
    with nidaqmx.Task() as ao_task:
        ao_task.ao_channels.add_ao_voltage_chan(f"{device}/{ao_channel}")

        ao_task.timing.cfg_samp_clk_timing(
            rate=sample_rate,
            sample_mode=AcquisitionType.CONTINUOUS,
            samps_per_chan=buffer_size
        )

        ao_task.out_stream.regen_mode = RegenerationMode.DONT_ALLOW_REGENERATION
        ao_task.out_stream.output_buf_size = buffer_size
        ao_task.out_stream.relative_to = WriteRelativeTo.CURRENT_WRITE_POSITION

        # Generate initial buffer
        initial_data = np.sin(2 * np.pi * 100_000 * np.arange(buffer_size) / sample_rate)
        ao_task.write(initial_data)

        ao_task.start()

        stop_event = threading.Event()

        def refill_buffer(task, stop_event):
            idx = buffer_size
            fast_freq = 500_000  # 500 kHz sine wave for fast sine
            while not stop_event.is_set():
                data = np.sin((2 * np.pi * fast_freq) * np.arange(idx, idx + half_buffer) / sample_rate)
                task.write(data, auto_start=False)
                idx += half_buffer

        stream_thread = threading.Thread(target=refill_buffer, args=(ao_task, stop_event))
        stream_thread.start()

        try:
            print("Streaming waveform. Press Ctrl+C to stop.")
            while True:
                time.sleep(1)
        except KeyboardInterrupt:
            print("Stopping streaming.")
            stop_event.set()
            stream_thread.join()

# To run the streaming, uncomment the following line:
continuous_waveform_streaming()


Streaming waveform. Press Ctrl+C to stop.
Stopping streaming.


In [20]:
"""Example of analog output voltage generation.

This example demonstrates how to continuously generate an
analog output waveform by providing new data to the output buffer
as the task is running.

This example is useful if you want to generate a non-repeating waveform,
make updates on-the-fly, or generate a frequency that is not an
even divide-down of your sample clock. In this example,
the default frequency value is 17.0 to demonstrate that non-regenerative output
can be used to create a signal with a frequency that is not an even divide-down
of your sample clock.
"""

from typing import Tuple

import numpy as np
import numpy.typing

import nidaqmx
from nidaqmx.constants import AcquisitionType, RegenerationMode


def generate_wave(
    frequency: float,
    amplitude: float,
    sampling_rate: float,
    number_of_samples: int,
    phase_in: float = 0.0,
    use_sine: bool = True,
) -> Tuple[numpy.typing.NDArray[numpy.double], float]:
    """Generates a sine or square wave with a specified phase.

    Args:
        frequency: Specifies the frequency of the wave.
        amplitude: Specifies the amplitude of the wave.
        sampling_rate: Specifies the sampling rate of the wave.
        number_of_samples: Specifies the number of samples to generate.
        phase_in: Specifies the phase of the wave in radians.
        use_sine: If True, generate sine wave; if False, generate square wave.

    Returns:
        Indicates a tuple containing the generated data and the phase
        of the wave after generation.
    """
    duration_time = number_of_samples / sampling_rate
    duration_radians = duration_time * 2 * np.pi
    phase_out = (phase_in + duration_radians) % (2 * np.pi)
    t = np.linspace(phase_in, phase_in + duration_radians, number_of_samples, endpoint=False)

    if use_sine:
        data = amplitude * np.sin(frequency * t)
    else:
        data = amplitude * np.sign(np.sin(frequency * t))
    return (data, phase_out)

def main():
    """Generate a continuous voltage waveform using an analog output channel of a NI-DAQmx device.

    This function sets up a task to generate a continuous voltage waveform using the specified
    analog output channel of a NI-DAQmx device. It configures the sampling rate, number of samples,
    and regeneration mode of the task. It then enters a loop where it continuously generates a
    sine wave with a specified frequency, amplitude, and phase, and writes the waveform to the
    analog output channel.
    The loop continues until the user interrupts the program by pressing Ctrl+C.

    Args:
        None

    Returns:
        None
    """
    with nidaqmx.Task() as task:
        is_first_run = True
        sampling_rate = 3_000_000.0
        number_of_samples = 5_000_000
        task.ao_channels.add_ao_voltage_chan("PXI1Slot6/ao1")
        task.out_stream.regen_mode = RegenerationMode.DONT_ALLOW_REGENERATION
        task.timing.cfg_samp_clk_timing(sampling_rate, sample_mode=AcquisitionType.CONTINUOUS)

        actual_sampling_rate = task.timing.samp_clk_rate
        print(f"Actual sampling rate: {actual_sampling_rate:g} S/s")

        try:
            phase = 0.0
            use_sine = True
            print("Generating voltage continuously. Press Ctrl+C to stop.")
            while True:
                data, phase = generate_wave(
                    frequency=1_000_000,
                    amplitude=1.0,
                    sampling_rate=actual_sampling_rate,
                    number_of_samples=number_of_samples,
                    phase_in=phase,
                    use_sine=use_sine
                )
                task.write(data)
                # Alternate use_sine between True and False
                use_sine = not use_sine
                if is_first_run:
                    print('First data written')
                    is_first_run = False
                    task.start()
        except KeyboardInterrupt:
            pass
        finally:
            task.stop()


if __name__ == "__main__":
    main()

Actual sampling rate: 3.0303e+06 S/s
Generating voltage continuously. Press Ctrl+C to stop.
First data written


In [ ]:
import nidaqmx
import numpy as np
from nidaqmx.constants import AcquisitionType, RegenerationMode, WriteRelativeTo

task = nidaqmx.Task()

# Set up AO channel
task.ao_channels.add_ao_voltage_chan("PXI1Slot6/ao1")

task.timing.cfg_samp_clk_timing(
    rate=10000,
    sample_mode=AcquisitionType.FINITE,
    samps_per_chan=1000
)

# Triggers
task.triggers.start_trigger.cfg_dig_edge_start_trig('/PXI1Slot2/PFI0')
task.triggers.


Turning mirror on and of

In [ ]:
# Turning mirror on and off
device = 'PXI1Slot2'
ao_channel = "ao1"
with nidaqmx.Task() as ao_task:
    ao_task.ao_channels.add_ao_voltage_chan(f"{device}/{ao_channel}")
    ao_task.write(4.0)
    ao_task.start()
    time.sleep(5.0)
    ao_task.write(0)
    ao_task.stop()

In [90]:
# Output the waveform at 1 MHz, sampled at 10 MHz

import numpy as np
import nidaqmx
from nidaqmx.constants import AcquisitionType
from nidaqmx.stream_writers import AnalogSingleChannelWriter
import time

# Desired output frequency of the waveform
waveform_freq = 1_00_000  # 1 MHz
sample_rate = 1_000_000   # 10 MHz sample rate

duration = 1  # seconds (buffer duration)
t = np.arange(0, duration, 1/sample_rate)

# Example: 1 MHz sine wave, sampled at 10 MHz
sine_wave = np.sin(2 * np.pi * waveform_freq * t)
# You can use other waveforms as needed, e.g. square, triangle, etc.

waveform = sine_wave.astype(np.float64)
buffer_size = len(waveform)

def stream_waveform():
    with nidaqmx.Task() as ao_task:
        ao_task.ao_channels.add_ao_voltage_chan("PXI1Slot6/ao1")
        ao_task.timing.cfg_samp_clk_timing(
            rate=sample_rate,
            sample_mode=AcquisitionType.CONTINUOUS,
            samps_per_chan=buffer_size
        )

        writer = AnalogSingleChannelWriter(ao_task.out_stream, auto_start=False)
        writer.write_many_sample(waveform)
        ao_task.start()

        print("Streaming 1 MHz waveform sampled at 10 MHz. Press Ctrl+C to stop.")
        try:
            while True:
                # Refill the buffer to avoid underrun
                writer.write_many_sample(waveform)
                time.sleep(duration / 2)  # Sleep for half the duration of the buffer
        except KeyboardInterrupt:
            print("Stopping streaming.")
            ao_task.stop()

stream_waveform()


Streaming 1 MHz waveform sampled at 10 MHz. Press Ctrl+C to stop.
Stopping streaming.


Below is a successful implementation of waveform triggering at ~1MHz

In [20]:
import numpy as np
import nidaqmx
from nidaqmx.constants import AcquisitionType, Edge

fs = 2_000_000  # 1 MS/s
cycles = 63
samples_per_cycle = 2
num_samples = cycles * samples_per_cycle
waveform = np.tile([1.0, 0.0], cycles)


with nidaqmx.Task() as ao_task:
    ao_task.ao_channels.add_ao_voltage_chan("PXI1Slot6/ao1")
    ao_task.timing.cfg_samp_clk_timing(
        rate=fs,
        sample_mode=AcquisitionType.FINITE,
        samps_per_chan=num_samples
    )

    trigger_source = '/PXI1Slot2/PFI0' 
                                      
    ao_task.triggers.start_trigger.cfg_dig_edge_start_trig(
        trigger_source=trigger_source,
        trigger_edge=Edge.FALLING
    )

    ao_task.triggers.start_trigger.retriggerable = True

    ao_task.write(waveform)
    print(f"Waiting for trigger on {trigger_source} to output waveform...")
    ao_task.start()
    ao_task.wait_until_done(timeout=10.0)
    print("Waveform output complete.")
    ao_task.stop()

Waiting for trigger on /PXI1Slot2/PFI0 to output waveform...


DaqError: Wait Until Done did not indicate that the task was done within the specified timeout.

Increase the timeout, check the program, and make sure connections for external timing and triggering are in place.
Task Name: _unnamedTask<C>

Status Code: -200560

Need to try streaming data in and using this trigger to know when to get the next sample

In [ ]:
import numpy as np
import nidaqmx
from nidaqmx.constants import AcquisitionType, Edge, TriggerUsage

fs = 2_000_000  # 1 MS/s
cycles = 63
samples_per_cycle = 2
num_samples = cycles * samples_per_cycle
waveform = np.tile([1.0, 0.0], cycles)


with nidaqmx.Task() as ao_task:
    ao_task.ao_channels.add_ao_voltage_chan("PXI1Slot6/ao1")
    ao_task.timing.cfg_samp_clk_timing(
        rate=fs,
        sample_mode=AcquisitionType.FINITE,
        samps_per_chan=num_samples
    )

    trigger_source = '/PXI1Slot2/PFI0' 
                                      
    ao_task.triggers.start_trigger.cfg_dig_edge_start_trig(
        trigger_source=trigger_source,
        trigger_edge=Edge.FALLING
    )

    TriggerUsage.ADVANCE

    ao_task.triggers.start_trigger.retriggerable = True

    ao_task.write(waveform)
    print(f"Waiting for trigger on {trigger_source} to output waveform...")
    ao_task.start()
    ao_task.wait_until_done(timeout=10.0)
    print("Waveform output complete.")
    ao_task.stop()

Waiting for trigger on /PXI1Slot2/PFI0 to output waveform...


DaqError: Wait Until Done did not indicate that the task was done within the specified timeout.

Increase the timeout, check the program, and make sure connections for external timing and triggering are in place.
Task Name: _unnamedTask<F>

Status Code: -200560

---
TRYING SOMETHING NEW
---



In [185]:
import numpy as np
import nidaqmx
from nidaqmx import stream_writers
from nidaqmx.constants import AcquisitionType, WriteRelativeTo, WaitMode, Edge, TriggerUsage
import time

def generate_sine_wave(frequency, duration_time, amplitude=1):
    sampling_rate = 100_000
    number_of_samples = duration_time * sampling_rate
    duration_radians = duration_time * 2 * np.pi
    phase_in = 0.0
    t = np.linspace(phase_in, phase_in+duration_radians, number_of_samples, endpoint=False)
    return (amplitude * np.sin(frequency * t))

for i in range(10):
    with nidaqmx.Task() as ao_task:
        ao_task.ao_channels.add_ao_voltage_chan("PXI1Slot6/ao1")
        ao_task.out_stream.regen_mode = RegenerationMode.DONT_ALLOW_REGENERATION
        ao_task.timing.cfg_samp_clk_timing(100_000.0, sample_mode=AcquisitionType.FINITE, samps_per_chan=1000)
        #ao_task.out_stream.relative_to = WriteRelativeTo.CURRENT_WRITE_POSITION
        #ao_task.out_stream.wait_mode = WaitMode.POLL
        data = generate_sine_wave(1000, 1)
        #ao_task.out_stream.output_buf_size = 1000000
        stream = stream_writers.AnalogSingleChannelWriter(ao_task.out_stream, auto_start=True)
        stream.write_many_sample(data)
        ao_task.wait_until_done(timeout=10)
        ao_task.stop()


In [216]:
import numpy as np
import nidaqmx
from nidaqmx.constants import AcquisitionType, Edge, TriggerUsage
import time

def generate_sine_wave(frequency, sampling_rate, duration_sec, amplitude=1):
    t = np.linspace(0, duration_sec, int(sampling_rate * duration_sec), endpoint=False)
    return amplitude * np.sin(2 * np.pi * frequency * t)

# Parameters
frequency = 500000  # 1 kHz sine wave
sampling_rate = 1_000_000  # 1 MHz
duration_sec = 5  # 5 seconds
chunk_duration_sec = 63e-2  # 63 microseconds
chunk_size = int(sampling_rate * chunk_duration_sec)

# Generate full 5 second sine wave
all_data = generate_sine_wave(frequency, sampling_rate, duration_sec)

# Split into chunks of 63 microseconds
chunks = [all_data[i:i+chunk_size] for i in range(0, len(all_data), chunk_size)]

with nidaqmx.Task() as task:
    full_start = time.perf_counter()
    task.ao_channels.add_ao_voltage_chan("PXI1Slot6/ao1")
    trigger_source = '/PXI1Slot2/PFI0' 
                                      
    ao_task.triggers.start_trigger.cfg_dig_edge_start_trig(
        trigger_source=trigger_source,
        trigger_edge=Edge.FALLING
    )
    ao_task.triggers.start_trigger.retriggerable = True
    ao_task.timing.cfg_samp_clk_timing(1000000, sample_mode=AcquisitionType.FINITE, samps_per_chan=len(chunks[0]))

    for i in range(len(chunks)):
        #task_start = time.perf_counter()
        task.out_stream.regen_mode = RegenerationMode.DONT_ALLOW_REGENERATION
        #data = generate_sine_wave(50000, 500000, i)

        task.write(chunks[i])
        #print(f"Write time: {time.perf_counter() - task_start}")
        task.start()
        #print_start = time.perf_counter()
        task.wait_until_done()
        #task.stop()
        #print(f"Print time (us): {(time.perf_counter() - print_start) * 1e6}")
        
    print(f"Full print: {time.perf_counter() - full_start}")

DaqError: The generation has stopped to prevent the regeneration of old samples. Your application was unable to write samples to the background buffer fast enough to prevent old samples from being regenerated.

To avoid this error, you can do any of the following:
1. Increase the size of the background buffer by configuring the buffer.
2. Increase the number of samples you write each time you invoke a write operation.
3. Write samples more often.
4. Reduce the sample rate.
5. If your data transfer method is interrupts, try using DMA or USB Bulk. 
6. Reduce the number of applications your computer is executing concurrently. 

In addition, if you do not need to write every sample that is generated, you can configure the regeneration mode to allow regeneration, and then use the Position and Offset attributes to write the desired samples.
Task Name: _unnamedTask<191>

Status Code: -200290